Import Libraries

In [1]:
import pandas as pd
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

Download the dataset


In [2]:
!gdown --id 1x9ztpUDKnL1XQFnpvPfIip52KpmFs-Me

Downloading...
From: https://drive.google.com/uc?id=1x9ztpUDKnL1XQFnpvPfIip52KpmFs-Me
To: /content/clean_profile_data_all.csv
100% 628k/628k [00:00<00:00, 4.74MB/s]


In [3]:
dataset = pd.read_csv('clean_profile_data_all.csv')

In [4]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [5]:
model_name=[]
model_train_acc=[]
model_test_accuracy=[]
model_train_f1=[]
model_test_f1=[]
def get_result(model, X_train, X_test, Y_train, Y_test):
    sc = StandardScaler() 
    sc.fit(X_train)
    X_train = sc.transform(X_train)
    X_test = sc.transform(X_test) 
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    prob_test=pd.DataFrame(model.predict_proba(X_test))
    prob_train=pd.DataFrame(model.predict_proba(X_train))
    test_f1_score = f1_score(Y_test, y_pred,pos_label='accept')
    train_f1_score = f1_score(Y_train, y_train_pred,pos_label='accept')
    train_accuracy=accuracy_score(Y_train, y_train_pred)
    test_accuracy=accuracy_score(Y_test, y_pred)
    test_cm = confusion_matrix(Y_test, y_pred,labels=['accept','reject'])
    train_cm = confusion_matrix(Y_train, y_train_pred,labels=['accept','reject'])
    model_name.append(model)
    model_train_acc.append(train_accuracy)
    model_test_accuracy.append(test_accuracy)
    model_test_f1.append(test_f1_score)
    model_train_f1.append(train_f1_score)
    return [train_cm,test_cm,train_accuracy,test_accuracy,train_f1_score, test_f1_score, prob_train,prob_test, y_pred,y_train_pred, model,sc]

In [6]:
selected_university=["northeastern_university","clemson_university","george_mason_university","georgia_institiute_of_technology","illinois_institute_of_technology","kansas_state_university","north_carolina_state_university_raleigh","new_york_university","rochester_institute_of_technology","rutgers_university_new_brunswick","state_university_of_new_york_at_stony_brook","syracuse_university","texas_a_m_university_college_station","university_of_connecticut","university_of_colorado_boulder","university_of_florida","university_of_north_carolina_at_charlotte",

"university_of_texas_arlington",
"university_of_texas_dallas",
"worcester_polytechnic_institute"]

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
import pickle
import math
acc=[]
for university in selected_university:
    data = dataset[(dataset.university_name == university)]
    training, testing = train_test_split(data, test_size=0.25, random_state=5, stratify=data['status'])
    numerical_data = training.select_dtypes(include = ['int64','float','uint8'])
    categorical_data = training.select_dtypes(include = ['object'])
    categorical_features = categorical_data.columns.values
    numerical_features = numerical_data.columns.values
    numerical_features = numerical_data.columns.values[:-1]
    random_forest_model=RandomForestClassifier(n_estimators=10)
    randomforest_model_results=get_result(random_forest_model,training[numerical_features],testing[numerical_features],training['status'],testing['status'])
    print(university+' test_accuracy:',randomforest_model_results[3])
    acc.append(randomforest_model_results[3])
    # print('train_accuracy:',randomforest_model_results[2])
    # print('test_f1_score:',randomforest_model_results[5])
    # print('train_f1_score:',randomforest_model_results[4])
    # generate_cm_roc(randomforest_model_results)  
    rf_classifier_pkl_filename = university+'.pickel'
    random_forest_classifier_model_pkl = open(rf_classifier_pkl_filename, 'wb')
    pickle.dump(randomforest_model_results[10], random_forest_classifier_model_pkl)
    random_forest_classifier_model_pkl.close()
    # print(training.head())
print(sum(acc)/len(acc))

northeastern_university test_accuracy: 0.6763285024154589
clemson_university test_accuracy: 0.8387096774193549
george_mason_university test_accuracy: 0.7424242424242424
georgia_institiute_of_technology test_accuracy: 0.71875
illinois_institute_of_technology test_accuracy: 0.6513761467889908
kansas_state_university test_accuracy: 0.5789473684210527
north_carolina_state_university_raleigh test_accuracy: 0.782312925170068
new_york_university test_accuracy: 0.525
rochester_institute_of_technology test_accuracy: 0.7325581395348837
rutgers_university_new_brunswick test_accuracy: 0.6447368421052632
state_university_of_new_york_at_stony_brook test_accuracy: 0.5894039735099338
syracuse_university test_accuracy: 0.6230769230769231
texas_a_m_university_college_station test_accuracy: 0.7216494845360825
university_of_connecticut test_accuracy: 0.75
university_of_colorado_boulder test_accuracy: 0.813953488372093
university_of_florida test_accuracy: 0.7083333333333334
university_of_north_carolina_at_

In [9]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib
for university in selected_university:
  loaded_model = joblib.load( "./"+university+".pickel")
  print(university+" "+loaded_model.predict([[0.0, 0.0,  0.0,0.0,0.0,0.0,0.0]]))

['northeastern_university reject']
['clemson_university accept']
['george_mason_university accept']
['georgia_institiute_of_technology accept']
['illinois_institute_of_technology accept']
['kansas_state_university accept']
['north_carolina_state_university_raleigh reject']
['new_york_university reject']
['rochester_institute_of_technology accept']
['rutgers_university_new_brunswick reject']
['state_university_of_new_york_at_stony_brook accept']
['syracuse_university accept']
['texas_a_m_university_college_station reject']
['university_of_connecticut accept']
['university_of_colorado_boulder reject']
['university_of_florida accept']
['university_of_north_carolina_at_charlotte reject']
['university_of_texas_arlington accept']
['university_of_texas_dallas accept']
['worcester_polytechnic_institute accept']


In [ ]:
university = ['northeastern_university', 'carnegie_mellon_university',
       'clemson_university', 'george_mason_university',
       'georgia_institiute_of_technology',
       'illinois_institute_of_technology',
       'indiana_university_bloomington', 'kansas_state_university',
       'university_of_maryland_college_park',
       'michigan_technological_university',
       'north_carolina_state_university_raleigh', 'new_york_university',
       'rochester_institute_of_technology',
       'rutgers_university_new_brunswick',
       'state_university_of_new_york_at_stony_brook',
       'syracuse_university', 'texas_a_m_university_college_station',
       'university_of_connecticut', 'university_of_colorado_boulder',
       'university_of_california_irvine', 'university_of_florida',
       'university_of_north_carolina_at_charlotte',
       'university_of_southern_california',
       'university_of_texas_arlington', 'university_of_texas_austin',
       'university_of_texas_dallas', 'university_of_iowa',
       'university_of_cincinnati', 'worcester_polytechnic_institute']
import joblib
for university in selected_university:
  loaded_model = joblib.load( "/content/Machine-Learning-Graduate-Studuent-Admission-Predictor/Final Project _ Graduate Admission Predictor/Data/"+university+".pickel")
  print(university+" "+loaded_model.predict([[0.0, 0.0,  0.0,0.0,0.0,0.0,0.0]]))

In [ ]:
dataset.university_name.unique()



In [ ]:
def trainmodel(X_train,y_train,X_test,y_test):
  import keras
  from keras.models import Sequential
  from keras.layers import Dense# Neural network
  model = Sequential()
  model.add(Dense(16, input_dim=20, activation='relu'))
  model.add(Dense(12, activation='relu'))
  model.add(Dense(4, activation='softmax'))
  #Dependencies
  import keras
  from keras.models import Sequential
  from keras.layers import Dense# Neural network
  model = Sequential()
  model.add(Dense(16, input_dim=7, activation='relu'))
  model.add(Dense(12, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  history = model.fit(X_train, y_train, epochs=100, batch_size=64)
  y_pred = model.predict(X_test)
  #Converting predictions to label
  pred = list()
  for i in range(len(y_pred)):
      pred.append(np.argmax(y_pred[i]))
  #Converting one hot encoded test label to label
  test = list()
  for i in range(len(y_test)):
      test.append(np.argmax(y_test[i]))

In [ ]:
dataset_x_train=[]
dataset_x_test=[]
dataset_y_train=[]
dataset_y_test=[]

for university in selected_university:
    data = dataset[(dataset.university_name == university)]
    training, testing = train_test_split(data, test_size=0.25, random_state=5, stratify=data['status'])
    numerical_data = training.select_dtypes(include = ['int64','float','uint8'])
    categorical_data = training.select_dtypes(include = ['object'])
    categorical_features = categorical_data.columns.values
    numerical_features = numerical_data.columns.values
    numerical_features = numerical_data.columns.values[:-1]
    random_forest_model=RandomForestClassifier(n_estimators=10)
    result = training['status'].replace('reject', 0)
    result=result .replace('accept',1)    
    result_test=testing['status'].replace('reject',0)
    result_test=result_test.replace('accept',1)
    # trainmodel(training[numerical_features],result,testing[numerical_features],result_test)
    X_train=training[numerical_features]
    y_train=result
    X_test=testing[numerical_features]
    y_test=result_test
    dataset_x_train.append(X_train)
    dataset_x_test.append(X_test)
    dataset_y_train.append(y_train)
    dataset_y_test.append(y_test)



    import keras
    from keras.models import Sequential
    from keras.layers import Dense# Neural network
    model = Sequential()
    model.add(Dense(7, input_dim=7, activation='relu'))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(48,activation="relu"))
    model.add(Dense(24,activation="relu"))
    model.add(Dense(16,activation="relu"))
    model.add(Dense(8,activation="relu"))
    model.add(Dense(8,activation="relu"))
    model.add(Dense(8,activation="relu"))
    model.add(Dense(8,activation="relu"))
    model.add(Dense(8,activation="relu"))
    model.add(Dense(4,activation="relu"))
    model.add(Dense(2,activation="relu"))
    # model.add(Dense(8, activation='relu'))
    # model.add(Dense(6, activation='relu'))
    # model.add(Dense(5, activation='relu'))
    # model.add(Dense(4, activation='relu'))
    # model.add(Dense(3, activation='relu'))
    # model.add(Dense(2, activation='relu'))
    # model.add(Dense(12, activation='relu'))
    # model.add(Dense(12, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test,y_test),epochs=50, batch_size=64,verbose=1)
    y_pred = model.predict(X_test)
    #Converting predictions to label
    pred = list()
    for i in range(len(y_pred)):
        pred.append(0 if i<0.50 else 1)
    #Converting one hot encoded test label to label
    test = list()
    # for i in range(len(y_test)):
    #     test.append(np.argmax(y_test[i]))
    score, acc = model.evaluate(X_test, y_test,
                            batch_size=5)
    print('Test score:', score)
    print('Test accuracy:', acc)
    score, acc = model.evaluate(X_train, y_train,
                            batch_size=5)
    print('Train score:', score)
    print('Train accuracy:', acc)
    model.save(university+"_model.h5")
    from keras.models import load_model
    model = load_model(university+"_model.h5")
    print("pred",model.predict(X_test[:1]))
    # randomforest_model_results=get_result(random_forest_model,training[numerical_features],testing[numerical_features],training['status'],testing['status'])
    print(university+' test_accuracy:',randomforest_model_results[3])
    # print('train_accuracy:',randomforest_model_results[2])
    # print('test_f1_score:',randomforest_model_results[5])
    # print('train_f1_score:',randomforest_model_results[4])
    # generate_cm_roc(randomforest_model_results)  
    rf_classifier_pkl_filename = university+'.pickel'
    random_forest_classifier_model_pkl = open(rf_classifier_pkl_filename, 'wb')
    pickle.dump(randomforest_model_results[10], random_forest_classifier_model_pkl)
    random_forest_classifier_model_pkl.close()
    # print(training.head())
    break

In [ ]:
dataset_x_train=[]
dataset_x_test=[]
dataset_y_train=[]
dataset_y_test=[]

for university in selected_university:
    data = dataset[(dataset.university_name == university)]
    training, testing = train_test_split(data, test_size=0.25, random_state=5, stratify=data['status'])
    numerical_data = training.select_dtypes(include = ['int64','float','uint8'])
    categorical_data = training.select_dtypes(include = ['object'])
    categorical_features = categorical_data.columns.values
    numerical_features = numerical_data.columns.values
    numerical_features = numerical_data.columns.values[:-1]
    random_forest_model=RandomForestClassifier(n_estimators=10)
    result = training['status'].replace('reject', 0)
    result=result .replace('accept',1)    
    result_test=testing['status'].replace('reject',0)
    result_test=result_test.replace('accept',1)
    # trainmodel(training[numerical_features],result,testing[numerical_features],result_test)
    X_train=training[numerical_features]
    y_train=result
    X_test=testing[numerical_features]
    y_test=result_test
    dataset_x_train.append(X_train)
    dataset_x_test.append(X_test)
    dataset_y_train.append(y_train)
    dataset_y_test.append(y_test)





In [ ]:

# for university in dataset.university_name.unique():
#     data = dataset[(dataset.university_name == university)]
#     training, testing = train_test_split(data, test_size=0.25, random_state=5, stratify=data['status'])
#     numerical_data = training.select_dtypes(include = ['int64','float','uint8'])
#     categorical_data = training.select_dtypes(include = ['object'])
#     categorical_features = categorical_data.columns.values
#     numerical_features = numerical_data.columns.values
#     numerical_features = numerical_data.columns.values[:-1]
#     result = training['status'].replace('reject', 0)
#     result=result .replace('accept',1)    
#     result_test=testing['status'].replace('reject',0)
#     result_test=result_test.replace('accept',1)
#     datastat(result_test,university)
# selected_university=["northeastern_university","clemson_university","george_mason_university","georgia_institiute_of_technology","illinois_institute_of_technology","kansas_state_university","north_carolina_state_university_raleigh","new_york_university","rochester_institute_of_technology","rutgers_university_new_brunswick","state_university_of_new_york_at_stony_brook","syracuse_university","texas_a_m_university_college_station","university_of_connecticut","university_of_colorado_boulder","university_of_florida","university_of_north_carolina_at_charlotte",

# "university_of_texas_arlington",
# "university_of_texas_dallas",
# "worcester_polytechnic_institute"]

In [ ]:
selected_university


In [ ]:
training[:1]

In [ ]:
for university in selected_university:
    data = dataset[(dataset.university_name == university)]
    data = data[(data.status == "accept")]
    print(university+" "+str(sum(data["test_score_toefl"])/len(data["test_score_toefl"])))

In [ ]:
def datastat(result_test,university):
  tr=0
  fa=0
  for i in result_test:
    if(i==0):
      fa=fa+1
    else:
      tr=tr+1

  if((tr)/(len(result_test))>0.30  and (fa)/(len(result_test)) >0.30):
      print(university)
      print("SIze "+str(len(result_test)))
      print("True "+str((tr)/(len(result_test))) +" %")
      print("False"+str((fa)/(len(result_test))) + " %")


In [ ]:
type(training['status'])
result = training['status'].replace('reject', 0)
result .replace('accept',1)